In [ ]:
import requests
import pandas as pd
from census import Census
from api_keys import api_key
import matplotlib.pyplot as plt
from scipy import stats
from unidecode import unidecode
import re

In [ ]:

#Years to loop through 
years = [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2021, 2022, 2023]  # You can add more years here
dsource = 'acs'
dname = 'acs1'
# Data columns to fetch
cols = 'NAME,B25077_001E,B25058_001E,B15003_022E,B01003_001E,B23025_004E,B17001_002E,B23025_003E'  
# List of all state FIPS codes

state_fips_codes = ['01', '02', '04', '05', '06', '08', '09', '10', '11', '12', '13', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29',
    '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '53', '54', '55', '56']  

# Initialize an empty DataFrame to collect data
all_data = []

# Loop through each year
for year in years:
    
    # Loop through all states to fetch data for all counties
    for state in state_fips_codes:
        # Construct the data URL for querying each state and its counties for the current year
        data_url = f'https://api.census.gov/data/{year}/{dsource}/{dname}?get={cols}&for=county:*&in=state:{state}'

        # Send the GET request to the Census API
        response = requests.get(data_url)

        # Check if the request was successful (200 indicates request was successfully processed)
        if response.status_code == 200:
            # Parse the response JSON data
            data = response.json()
            
            # Create a DataFrame from the response data
            df = pd.DataFrame(data[1:], columns=data[0])  # Skip the header row
            
            # Rename columns
            df = df.rename(columns={
                'NAME': 'County Name',
                'state': 'State FIPS',
                'county': 'County FIPS',
                'B25077_001E': 'Median House Price ($)',
                'B25058_001E': 'Median Rent ($)',
                'B01003_001E': 'Total Population',
                'B15003_022E': 'Population with Bachelor\'s Degree',
                'B23025_004E': 'Number of Employed People',
                'B17001_002E': 'People with Income Below Poverty',
                'B23025_003E': 'Total Labor Force'
            })
            
            # Add a column for the year
            df['Year'] = year
            
            # # Split the 'County Name' column into 'County' and 'State'
            # df[['County', 'State']] = df['County Name'].str.split(',', expand=True)
            
            # # Optionally, clean up any leading/trailing spaces from the new columns
            # df['County'] = df['County'].str.strip()
            # df['State'] = df['State'].str.strip()
            
            # # Drop the original 'County Name' column
            # df = df.drop(columns=['County Name'])
            
            # Append the data for this year and state to the list
            all_data.append(df)
        else:
            print(f"Error: {response.status_code} for State: {state} in Year: {year}")

# Concatenate all data into a single DataFrame
clean_data_df = pd.concat(all_data, ignore_index=True)

# Save the resulting DataFrame to a CSV file
clean_data_df.to_csv('census_data_all_year.csv', index=False)

print("Data saved to 'census_data_all_year.csv'")


In [ ]:

# Convert columns to numeric, errors='coerce' will turn invalid values into NaN
numeric_columns = [
    'Median House Price ($)', 'Median Rent ($)', 'Total Population',
    'Population with Bachelor\'s Degree', 'Number of Employed People',
    'People with Income Below Poverty', 'Total Labor Force'
]

# Convert all columns to numeric
for col in numeric_columns:
    clean_data_df[col] = pd.to_numeric(clean_data_df[col], errors='coerce')

# Now print the min and max values, ignoring NaNs
print('Minimum Values:')
print(clean_data_df.min())
print('---------------------------------------------------')
print('Maximum Values:')
print(clean_data_df.max())

final_df = clean_data_df.dropna()

# Optionally, if you want to handle NaNs by filling them with a specific value:
#clean_data_df.fillna(0, inplace=True)  # Fill NaN values with 0

# Check for missing values and data types
print('Number of Null Values in Each Column:')
print(clean_data_df.isnull().sum())

# Removes Non-ASCII Characters
clean_data_df['County Name Cleaned'] = df['County Name'].apply(lambda x: unidecode(x).strip())

In [ ]:
clean_data_df.to_csv('census_data_all.csv', index=False)

In [ ]:
clean_data_df['Annual Rent-to-Price Ratio'] = ((clean_data_df['Median Rent ($)']*12) / clean_data_df ['Median House Price ($)']) * 100
clean_data_df['Employment Rate %'] = (clean_data_df['Number of Employed People'] / clean_data_df['Total Labor Force']) * 100
clean_data_df['% People Living in Poverty'] = (clean_data_df['People with Income Below Poverty'] / clean_data_df['Total Population']) * 100

clean_data_df

In [ ]:
#GROUPING ALL DATA BY COUNTIES
#To ensure exact matches, we need to group by all State FIPS, County FIPS, and County Name.
#This is because counties in different states may have the same name.
clean_data_df['Combined FIPS'] = clean_data_df['County FIPS'] + clean_data_df['State FIPS']

grouped_df = clean_data_df.groupby(['County Name', 'Combined FIPS'])
grouped_df.head()